# POC Power BI TS Formation

In [253]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import os
from os import path
import glob
import xlsxwriter
import openpyxl # 3.0
import re
import datetime as dt

## 1) Déclaration de la classe Datamanagement et des fonctions associées

In [254]:
class Datamanagement:
    def __init__(self):
        self.source = {} # Dictionnaire contenant le nom du fichier source et son contenu stocké dans un DataFrame
        self.dim = {} # Dictionnaire contenant le nom et les dataframes des tables de dimension
        self.fact = {} # Dictionnaire contenant le nom et les dataframes liés aux tables de faits
        self.changed_keys = {}        
        self.group_col = pd.DataFrame(columns=["Nom colonne", "Dimension cible", "Colonne cible"], data=[]) # DataFrame contenant les colonnes de regroupement créés
        try:
            self.settings = pd.ExcelFile(r"Settings & documentation\Settings.xlsx")  # Fichier setting dans un dataframe
            self.setting_sheets = self.settings.sheet_names # List des onglets de setting
        except:
            pass


    def import_csv(self, filename, engine_val=None, encoding_val='utf-8', sep_val=';',low_memory_val=False):
        self.source[filename] = pd.read_csv(r'Data/'+ filename + '.csv', engine=engine_val, encoding=encoding_val, sep=sep_val, low_memory=low_memory_val)
        return self.source[filename]
    
    def import_xlsx(self, filename, sheetname=0):
        self.source[filename] = pd.read_excel(r'Data/'+ filename + '.xlsx', sheet_name = sheetname)
        return self.source[filename]
    
    def update_key(self, changed_keys, filename):
        self.changed_keys[filename] = changed_keys
        self.source[filename].rename(columns=changed_keys , inplace=True)
        return self.source[filename]
    
    def import_dim(self,dimname, dimdataframe):
        self.dim[dimname]=dimdataframe
        
    def import_fact(self,factname, factdataframe):
        self.fact[factname] = factdataframe
    
    def export(self):
        for key, value in self.dim.items():
            location = "Transformed data"
            file_name = str(key) + '.csv'
            location = os.path.join(location, file_name)  
            value.to_csv(location, encoding='utf-8', index=False)
            
        for key, value in self.fact.items():
            location = "Transformed data"
            file_name = str(key) + '.csv'
            location = os.path.join(location, file_name)  
            value.to_csv(location, encoding='utf-16', index=False)
            
    def export_audit(self):
        for key, value in self.dim.items():
            location = "Transformed data"
            file_name = str(key) + '.xlsx'
            location = os.path.join(location, file_name)  
            value.to_excel(location, index=False)
            
        for key, value in self.fact.items():
            location = "Transformed data"
            file_name = str(key) + '.xlsx'
            location = os.path.join(location, file_name)  
            value.to_excel(location, index=False)
            
    def import_data(self):
# Fonction générant le fichier setting lors du premier chargement si ce dernier n'existe pas
# Etape 1 lecture des fichiers présents dans data et extraction des colonnes ainsi que des informations liées à la qualité de données
# Etape 2 vérification si le fichier Settings.xlsx existe, s'il existe, il faut éventuellement le modifier avec les nouvelles informations, sinon le créer
        
        df_files_col = pd.DataFrame()
        p = 1

        #Etape 1 lecture des fichiers présents dans data et extraction des colonnes ainsi que des informations liées à la qualité de données
        for key, value in self.source.items():
            df = value

            for i, col in enumerate(df.columns):              
                df_files_col.at[p, 'Nom fichier source'] = key
                df_files_col.at[p,'Nom champ source'] = col
                df_files_col.at[p,'Type'] = str(df[col].dtypes)
                df_files_col.at[p,'Synthèse'] = str(df[col].describe())
                df_files_col.at[p, "Date ajout"] = str(datetime.date(datetime.now()))
                df_files_col.at[p, "Date modification"] = str(datetime.date(datetime.now()))
                
                if i != 0:
                    p = p + i
                else:
                    p += 1

        df_dir_files = df_files_col.copy()
        df_dir_files = df_dir_files[["Nom fichier source", "Date ajout", "Date modification"]].drop_duplicates()

        if path.exists("Settings & documentation\Settings.xlsx"):
            pass

        else:
            writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='xlsxwriter')
            workbook  = writer.book
            df_dir_files.to_excel(writer, sheet_name='Fichiers Source', index=False)
            df_files_col[['Nom fichier source', 'Nom champ source', 'Type', 'Synthèse']].to_excel(writer, sheet_name='Fichiers et colonnes source', index=False)

            worksheet_FCS = writer.sheets["Fichiers et colonnes source"]
            worksheet_FS = writer.sheets["Fichiers Source"]
            #worksheet1 = workbook.add_worksheet("Mapping données")

            cell_format_FCS = workbook.add_format() 
            cell_format_FCS.set_text_wrap()
            cell_format_FCS.set_align('center')
            cell_format_FCS.set_align('vcenter')

            cell_format_FS = workbook.add_format()    
            cell_format_FS.set_align('left')

            worksheet_FCS.set_column('A:B', 30, cell_format_FCS)
            worksheet_FCS.set_column('C:C', 15, cell_format_FCS)
            worksheet_FCS.set_column('D:D', 25, cell_format_FCS)
            worksheet_FS.set_column('A:A', 40, cell_format_FS)
            worksheet_FS.set_column('B:B', 20, cell_format_FS)
            worksheet_FS.set_column('C:C', 20, cell_format_FS)
            writer.save()
            workbook.close()
        self.settings = pd.ExcelFile(r"Settings & documentation\Settings.xlsx")  # Fichier setting dans un dataframe
        self.setting_sheets = self.settings.sheet_names # List des onglets de setting
    def add_sheet_mapcol(self,DicDataframe=None):
# Fonction créant la feuille Mapping dim dans le fichier Settings
# Cette feuille contient l'ensemble des valeurs de dimension, elle permet de réaliser un mapping pour changer le nom des attributs

        if DicDataframe == None:
            DicDataframe = self.dim           
            
        df = pd.DataFrame(columns=["Nom dimension", "Nom colonne"],data=[])

        for key, value in DicDataframe.items():        

            for i, col in enumerate(value.columns):
                df.at[i, "Nom dimension"] = key
                df.at[i, "Nom colonne"] = col
                df.at[i, "Nouveau nom"] = ""
                df.at[i, "A mapper"] = "Non"                

        workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")  
        writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
        writer.book = workbook1
        df.to_excel(writer, sheet_name="Mapping dim colonne",engine='openpyxl',index=False)     
        writer.save()
        writer.close()

        workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")
        writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
        writer.book = workbook1
        workbook1["Mapping dim colonne"].column_dimensions["A"].width = 20
        workbook1["Mapping dim colonne"].column_dimensions["B"].width = 20
        workbook1["Mapping dim colonne"].column_dimensions["C"].width = 20

        writer.save()
        writer.close()
    
    def map_col(self):        
# Fonction transformant le nom des colonnes suivant le mapping effectué dans l'onglet Mapping dim colonne du fichier setting
# Entrée: Dictionnaire key: Nom dataframe  Value : Dataframe
# Sortie un dictionnaire "nom"/Dataframe avec les valeurs des colonnes actualisées suivant l'onglet de mapping "Mapping dim colonne" du fichier settings

        dict_dataframe = self.dim
        md = pd.read_excel(r"Settings & documentation\Settings.xlsx", sheet_name="Mapping dim colonne")
        mdf = md["Nom dimension"].unique()
        dic_map = {}
        
        for dim in mdf:
            map = md.loc[(md["Nom dimension"] == dim ) & (md["Nouveau nom"].notnull()== True),["Nom colonne","Nouveau nom"]]
            map.reset_index(drop=True, inplace=True)

            for i, val in enumerate(map.iterrows()):           
                dict_dataframe[dim].rename(columns={map.at[i, "Nom colonne"]:map.at[i,"Nouveau nom"]},inplace=True)

        my_dim = dict_dataframe    
    
    def export_to_settings(self, dataframe, sheetname, position = None):
# Fonction permettant d'exporter un dataframe vers un onglet de setting avec un formatage automatique des colonnes
# Paramètres: 
# dataframe: correspond au dataframe à importer,
# sheetname: correspond au nom de la feuille cible qui sera remplacées,
# position: permet de choisir la position de la feuille modifiée dans le fichier excel
   
        letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
        numbers = range(0,26,1)
        dic_letters = {} # Dictionnaire contenant les lettres de l'alphabet    
        val_max = [] # Liste contenant la taille des valeurs les plus longues

        # Création d'un dictionnaire stockant les lettres de l'alphabet et leur position afin d'alimenter les fonctions openpyxl
        for i in range(len(numbers)): 
            dic_letters[numbers[i]] = letters[i]

        # Alimentation de val_max
        for col in dataframe.columns:
            max = (dataframe[col].astype(str)).str.len().max()        
            if max != 0:
                val_max.append(int(max)+1)
            else:
                val_max.append(15)

       # Ouverture du fichier setting et création de l'onglet sheetname
    
        workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")  
        writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
        writer.book = workbook1
        list_sheet = workbook1.get_sheet_names() # Récupère la liste des feuilles

        if sheetname in list_sheet: # Si l'onglet existe déjà on va le supprimer pour le remplacer
            std = workbook1.get_sheet_by_name(sheetname)
            workbook1.remove_sheet(std)

        dataframe.to_excel(writer, sheet_name= sheetname, engine='openpyxl', index=False)     
        writer.save()
        writer.close()

        # Gère la position de la feuille

        workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")  
        writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
        writer.book = workbook1

        if position == None:        
            pos = len(list_sheet) - 1
        else:
            pos = position

        sheets = workbook1._sheets

        sheet = sheets.pop(len(list_sheet) - 1)
        sheets.insert(pos, sheet)

        writer.save()
        writer.close()

       # Réouverture du fichier setting
    
        workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")  
        writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
        writer.book = workbook1

        # Alimentation du fichier excel avec les données du dataframe

        for i, width in enumerate(val_max):
            if width < 11:
                workbook1[sheetname].column_dimensions[dic_letters[i]].width = 12
            else:
                workbook1[sheetname].column_dimensions[dic_letters[i]].width = width

        # Sauvegarde et fermeture du fichier excel
        
        writer.save()
        writer.close()    
    
    def mapp_data(self):
        result = pd.DataFrame()
        test=1

        try:       
            to_mapp_data = pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name="Mapping données") # Lecture de la feuille Mapping données
            to_mapp_data = to_mapp_data.loc[to_mapp_data["Valeurs cible"].isna() == False]
            to_mapp_data.reset_index(drop=True, inplace=True)

            if to_mapp_data.empty == False:
                for i, val in enumerate(to_mapp_data.iterrows()):
                    self.dim[to_mapp_data.at[i,"Table"]][to_mapp_data.at[i,"Colonnes"]].replace(to_replace =to_mapp_data.at[i,"Valeurs actuelles"], 
                     value = to_mapp_data.at[i,"Valeurs cible"], inplace=True)
        except:
                mapp_col = pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name="Mapping dim colonne")
                if mapp_col.empty == False:
                    mapp_col = mapp_col.loc[mapp_col["A mapper"] == "Oui", ["Nom dimension", "Nom colonne", "Nouveau nom"]]
                    mapp_col["Colonne"] = ""
                    mapp_col.reset_index(inplace=True, drop=True)
                    mapp_val = pd.DataFrame(columns=["Table", "Colonnes", "Valeurs actuelles", "Valeurs cible"])
                    p=1
                    if mapp_col.empty == False:
                        for i, val in enumerate(mapp_col.iterrows()):
                            if str(mapp_col.at[i, "Nouveau nom"]) != "nan":
                                mapp_col.at[i,"Colonne"] = mapp_col.at[i, "Nouveau nom"]
                            else:
                                 mapp_col.at[i,"Colonne"] = mapp_col.at[i, "Nom colonne"]

                        for i,val in enumerate(mapp_col[["Nom dimension", "Colonne"]].iterrows()):

                            if (self.dim[mapp_col.at[i, "Nom dimension"]][mapp_col.at[i, "Colonne"]]).empty == False:
                                list_val = (self.dim[mapp_col.at[i, "Nom dimension"]][mapp_col.at[i, "Colonne"]]).unique()
                                nom_col = mapp_col.at[i, "Colonne"]
                                nom_table = mapp_col.at[i, "Nom dimension"]

                            if len(list_val) != 0:
                                for i, val in enumerate(list_val):                           
                                    mapp_val.at[p,"Table"] = nom_table
                                    mapp_val.at[p,"Colonnes"] = nom_col
                                    mapp_val.at[p,"Valeurs actuelles"] = val
                                    mapp_val.at[p,"Valeurs cible"] = ""
                                    p += 1

                    if mapp_val.empty: 
                        pass            
                    else:
                        self.export_to_settings(mapp_val,"Mapping données")

    def __compare__(self,sheet_name, cible_dataframe, excl_col= [], position=None):        
        source = pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name=sheet_name)
        source["compare"] = ""
        cible = cible_dataframe
        cible["compare"] = ""
        colonnes_compared= [col for col in list(source.columns) if col not in excl_col]
        
        for col in colonnes_compared:
            source["compare"] = source["compare"] + source[col].astype('str')        
        
        for col in colonnes_compared:
            cible["compare"] = cible["compare"] + cible[col].astype('str')
        
        to_keep = [line for line in list(source["compare"]) if line in list(cible["compare"])]
        
        
        to_add = [line for line in list(cible["compare"]) if line not in list(source["compare"])]
        
        result = source.loc[source["compare"].isin(to_keep)]

        
        if to_add != "":
            result = result.append(cible.loc[cible["compare"].isin(to_add)])
            
        result.reset_index(drop=True, inplace=True)
        result.drop(["compare"], axis=1, inplace=True)
        return result
        #self.export_to_settings(result,sheet_name,position)
    
    def update_fichier_source(self):
        #Fonction mettant à jour l'onglet "Fichiers Source" du fichier Setting
        
        df_files_col = pd.DataFrame()
        p = 1

        #lecture des fichiers présents dans data et extraction des noms des fichiers     
        for key, value in self.source.items():
            
            df = value

            for i, col in enumerate(df.columns):
                df_files_col.at[p, 'Nom fichier source'] = key
                df_files_col.at[p,'Nom champ source'] = col
                df_files_col.at[p, "Date ajout"] = ""
                df_files_col.at[p, "Date modification"] = ""
                p = p + i

        df_dir_files = df_files_col.copy()
        df_dir_files = df_dir_files[["Nom fichier source", "Date ajout", "Date modification"]].drop_duplicates()
        result = self.__compare__("Fichiers Source", df_dir_files, ["Date ajout", "Date modification"],0 )
        result["Date ajout"] = result["Date ajout"].fillna(str(datetime.date(datetime.now())))
        result["Date modification"] = str(datetime.date(datetime.now()))
        
        self.export_to_settings(result,"Fichiers Source",0)
        
    def update_columns(self):
        #Fonction mettant à jour l'onglet Mapping dim colonne suite à l'ajout des nouvelles dimensions
        existing_colmap =  pd.read_excel(r"Settings & documentation\Settings.xlsx", sheet_name='Mapping dim colonne') # Contient les valeurs de mapping dim colonne
        result = pd.DataFrame()
        if existing_colmap.empty == True:
            pass
        else:
            fresh_col = pd.DataFrame(columns=["Nom dimension", "Nom colonne"], data=[]) # Contient l'ensemble des nouvelles valeurs pour Mapping dim colonne
            existing_colmap["Comparaison"] = existing_colmap["Nom dimension"] + existing_colmap["Nom colonne"] # Création de la colonne "Comparaison" servant à comparer les lignes
            to_keep = [] # Contient la liste des valeurs de la colonne "Comparaison" à conserver dans existing_colmap
            to_add = [] # Contient la liste des valeurs de la colonne "Comparaison" à rajouter
            p=0

            for key, value in self.dim.items():

                for i, col in enumerate(value.columns):
                    fresh_col.at[p, "Nom dimension"] = key
                    fresh_col.at[p, "Nom colonne"] = col
                    fresh_col.at[p, "Nouveau nom"] = ""
                    fresh_col.at[p, "A mapper"] = "Non"
                    p+=1

            fresh_col["Comparaison"] = fresh_col["Nom dimension"] + fresh_col["Nom colonne"]
            to_keep = list(fresh_col["Comparaison"])
            existing_colmap = existing_colmap.loc[existing_colmap["Comparaison"].isin(to_keep)]
            existing_colmap.reset_index(drop=True, inplace=True)

            # On détermine la liste des nouvelles valeurs absentes de la feuille de mapping

            for val in to_keep:
                if val in list(existing_colmap["Comparaison"]):
                    pass

                else:
                    to_add.append(val)

            fresh_col = fresh_col.loc[fresh_col["Comparaison"].isin(to_add)]
            fresh_col.reset_index(drop=True, inplace=True)

            result = existing_colmap.append(fresh_col)

            # On exporte le résultat dans l'onglet Mapping dim colonne de Setting
            result = result.drop(["Comparaison"], axis=1)
            result = result.sort_values(["Nom dimension", "Nom colonne"])
            self.export_to_settings(result, "Mapping dim colonne",2)

        return result
    
    def update_map_val(self):       
        mapp_col = pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name="Mapping dim colonne")
        mapp_col = mapp_col.loc[mapp_col["A mapper"] == "Oui", ["Nom dimension", "Nom colonne", "Nouveau nom"]]
        mapp_col["Colonne"] = ""
        mapp_col.reset_index(inplace=True, drop=True)
        mapp_val = pd.DataFrame(columns=["Table", "Colonnes", "Valeurs actuelles", "Valeurs cible"])
        p=1
        if mapp_col.empty == False:
            for i, val in enumerate(mapp_col.iterrows()):
                if str(mapp_col.at[i, "Nouveau nom"]) != "nan":
                    mapp_col.at[i,"Colonne"] = mapp_col.at[i, "Nouveau nom"]
                else:
                     mapp_col.at[i,"Colonne"] = mapp_col.at[i, "Nom colonne"]        

            for i,val in enumerate(mapp_col[["Nom dimension", "Colonne"]].iterrows()):

                if (self.dim[mapp_col.at[i, "Nom dimension"]][mapp_col.at[i, "Colonne"]]).empty == False:
                    list_val = (self.dim[mapp_col.at[i, "Nom dimension"]][mapp_col.at[i, "Colonne"]]).unique()
                    nom_col = mapp_col.at[i, "Colonne"]
                    nom_table = mapp_col.at[i, "Nom dimension"]

                if len(list_val) != 0:
                    for i, val in enumerate(list_val):                           
                        mapp_val.at[p,"Table"] = nom_table
                        mapp_val.at[p,"Colonnes"] = nom_col
                        mapp_val.at[p,"Valeurs actuelles"] = val
                        mapp_val.at[p,"Valeurs cible"] = ""
                        p += 1
                        
        if mapp_val.empty: 
            pass     
        else:
            result = self.__compare__("Mapping données", mapp_val,excl_col=["Valeurs cible"])
            self.export_to_settings(result,"Mapping données",3)
            
    def add_col_group(self, regroup_name, dim_source, col_source):
            
        target_sheet = pd.DataFrame(columns=["Dimension source", "Colonne source", "Valeurs sources", "Valeurs cibles"], data=[])
        export_name =  "Gr_" + dim_source + "_" + regroup_name 
        
        # Préparation du dataframe target_sheet
        
        for i, val in enumerate(list(self.dim[dim_source][col_source].unique())):
            target_sheet.at[i, "Dimension source"] = dim_source
            target_sheet.at[i, "Colonne source"] = col_source
            target_sheet.at[i, "Valeurs sources"] = val
        
        if export_name in self.setting_sheets:
            result = self.__compare__(export_name, target_sheet, excl_col= ["Valeurs cibles"])
            self.export_to_settings(result,export_name)
            replaced_values = list(result.loc[result["Valeurs cibles"].isna() == False, "Valeurs sources"])
            replacing_values = list(result.loc[result["Valeurs cibles"].isna() == False, "Valeurs cibles"])
            self.dim[dim_source][export_name] = self.dim[dim_source][col_source]
            self.dim[dim_source][export_name].replace(replaced_values, replacing_values, inplace=True)
        else:
            self.export_to_settings(target_sheet, export_name)                
        
        

### Instanciation de la classe Datamanagement

In [255]:
my_data = Datamanagement()

## 2) Import des données

In [256]:
%%time
### Ajouter les tables à intégrer ici
temps = my_data.import_xlsx("Dim temps")
offres = my_data.import_xlsx('PBI_Offres')
candidature = my_data.import_xlsx('PBI_Candidatures')
#tcw = my_data.import_csv('191119_TrainingCollectiveWishes') # Demandes collectives non affectées à un plan de formation

Wall time: 40.6 s


### Initialisation du fichier setting ou update du fichier setting

In [257]:
my_data.import_data()
my_data.update_fichier_source()

C:\Users\sgasmi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:201: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
C:\Users\sgasmi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:204: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
C:\Users\sgasmi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:205: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).


## 3) Transformation des données sources

In [258]:
%%time
# Remarques: Les données offres comportent des doublons. Une offre est publiée par exemple plusieurs fois le même jour, parfois même à la même minute.
# Des règles sont appliquées afin de rendre unique la combinaison IdDemandeAuto, Date de publication de l'offre, Date de clôture réelle

# Ajout du champ date de candidature contenant uniquement la partie date du champ Date de publication de l'offre
offres["date de publication"] = offres["Date de publication de l'offre"].dt.date

# Suppression du champ "Actuellement publié" qui dédoublonne les lignes inutilement
offres = offres.drop(["Actuellement publié"], axis=1)

#Suppression des lignes doublons
offres = offres.drop_duplicates()

# Suppression des Date de publication identiques (on conserve pour un même jour que l'heure la plus récente ): regroupement des données par "IdDemandeAuto","date candidature" et max "Date de publication de l'offre" afin ne conserver que les candidatures les plus récentes chauque jour
to_keep = offres.groupby(["IdDemandeAuto","date de publication"])["Date de publication de l'offre"].max().reset_index()
to_keep = to_keep.merge(offres[["IdDemandeAuto", "date de publication","Date de publication de l'offre","Date de clôture réelle"]], on=["IdDemandeAuto","date de publication","Date de publication de l'offre"], how="left")

# Lorsque date et heure de candidature identiques, on conserve la ligne avec la date de clôture réelle la plus grande
to_keep = to_keep.groupby(["IdDemandeAuto","date de publication","Date de publication de l'offre"])["Date de clôture réelle"].max().reset_index()

# Jointure avec la table offres afin de récupérer les autres données
offres = to_keep.merge(offres, on=["IdDemandeAuto","date de publication","Date de publication de l'offre","Date de clôture réelle"], how="left")
offres.reset_index(drop=True, inplace=True)


Wall time: 217 ms


In [259]:
# La fonction indicator permet de rattacher un évènement temporel (changement de statut, date de publication) à la dernière date du mois durant laquelle l'évènement s'est produit.
# Ceci permet de réaliser des jointures dans la table de fait pour rattacher l'évènement à une ligne.

def indicator_dt(dataframe_source, champ_source, nom_champ_cible):
    dataframe_source[nom_champ_cible] = dataframe_source[champ_source].apply(lambda x: x.replace(day=x.days_in_month))
    dataframe_source[nom_champ_cible] = dataframe_source[nom_champ_cible].astype(str)
    dataframe_source[nom_champ_cible] = dataframe_source[nom_champ_cible].apply(lambda x: x[:10])
    

indicator_dt(offres, "Date de publication de l'offre", "Ind dt publication de l'offre")
indicator_dt(offres, "Date de clôture réelle", "Ind clôture réelle")
indicator_dt(offres, "Date d'ouverture", "Ind date d'ouverture")
indicator_dt(offres, "Date de fermeture", "Ind date de fermeture")
offres["custom"] = 1


offres["publication"] = 1
offres["clôture"] = 1
offres["ouverture"] = 1
offres["fermeture"] = 1


my_data.import_dim("offres_test", offres)


In [260]:
indicator_dt(candidature,"Date du dépôt de candidature" , "Ind dépôt candidature")
candidature["candidature"] = 1

## 5) Creation des tables de dimension

### Création de la table dimension offres

In [261]:
%%time
# Génération de la table de dimension offre contenant les attributs d'offre n'évoluant peu

# Etape 1: Sélection des champs concernés qui seront stockés dans un DataFrame (équivalent d'une table)
dim_offres = offres[["IdDemandeAuto","Pays","Domaine professionnel","Branche","Recrutement interne/ externe","Lieu de travail","Intitulé du poste","Langue","Employeur (si connu)","Région","NP","Type d'emploi","Niveau de formation","EO","niveau d’expérience requis)","Line Manager",
                     "Talent Developer","Bibliothèque d'emplois","Période d'arrivée souhaitée","Localisation (Précisions/Mots-clés)","Nom complet","Mail Adresse TD","Adresse mail equipe d'offre","Type de contrat","Contrat rotationnel","Marque"]].copy() 

print(offres["Date de publication de l'offre"].apply(lambda x: x.replace(day=x.days_in_month))) # Récupération du dernier jour du mois

# Etape 2: Suppression des doublons et réinitialisation de l'index
dim_offres.drop_duplicates(inplace=True)
dim_offres.reset_index(drop=True, inplace=True)

#Etape 3: Ajout d'une clé technique
dim_offres["Key_offres"] = dim_offres.index
my_data.import_dim("dim_offres", dim_offres)


0       2017-11-30 14:33:00
1       2017-11-30 14:33:00
2       2017-12-31 14:33:00
3       2017-11-30 08:44:00
4       2018-02-28 08:48:00
                ...        
22975   2017-12-31 09:29:00
22976   2018-01-31 13:57:00
22977   2018-03-31 13:57:00
22978   2018-04-30 13:57:00
22979   2014-12-31 13:59:00
Name: Date de publication de l'offre, Length: 22980, dtype: datetime64[ns]
Wall time: 210 ms


### Création de la dimension temps

In [262]:
%%time
# Création de la table de temps. Nous gardons que les 2 dernières années
dim_temps =  temps[["date key","full date", "day num in month", "month", "month name", "year"]].copy()

# Détermination de l'année actuelle
now = dt.datetime.now()
actual_year = now.year
now = np.datetime64(now)

# dim_temps est filtrée sur l'année courante et l'année précédente
dim_temps = dim_temps.loc[(dim_temps["year"]==actual_year) | (dim_temps["year"] == actual_year-1) | (dim_temps["year"] == actual_year-2)]

# Les dates portants sur le furtur ne sont pas retenues sur l'année en cours
dim_temps = dim_temps.loc[dim_temps["full date"] <= now ]
my_data.import_dim("Temps", dim_temps)

# Création de la table fact_temps permettant de créer la table de fait
fact_temps = dim_temps[["year", "month","day num in month"]]
fact_temps = fact_temps.groupby(["year", "month"])["day num in month"].max().reset_index()

fact_temps = fact_temps.merge(dim_temps[["year", "month", "day num in month", "full date", "date key"]], how='left', on=["year", "month","day num in month"])
fact_temps["custom"] = 1


Wall time: 9.99 ms


### Traitement du mapping des colonnes et des valeurs

In [263]:
%%time

#Verification si la colonne Mapping dim existe dans le fichier setting, si non on va la créer
try:
    pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name="Mapping dim colonne")   
    Mapp_sheet = True    
except:
    Mapp_sheet = False
    
    
if Mapp_sheet == True:
    my_data.update_columns()
    pass
else:    
    #add_sheet_mapcol(my_data.dim)
    my_data.add_sheet_mapcol()

my_data.map_col()
#my_data.dim = map_col(my_data.dim)

my_data.mapp_data()

try:
    pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name="Mapping données")   
    Mapp_val = True    
except:
    Mapp_val = False
    
if Mapp_val == True :
    my_data.update_map_val()

#my_data.add_col_group("group1", "Plan de formation", "planname")

Wall time: 296 ms


C:\Users\sgasmi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:201: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
C:\Users\sgasmi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:204: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
C:\Users\sgasmi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:205: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).


## 6) Création des tables de fait

### Table de faits offres d'emplois

In [264]:
%%time

# Création de la table de fait offres d'emploi: Grain = 1 offre par mois

fact_offres = offres[["IdDemandeAuto","custom"]].copy()
fact_offres = fact_offres.drop_duplicates()
fact_offres = fact_offres.merge(fact_temps[["date key","full date","custom"]], on=["custom"], how = 'outer')
fact_offres["full date"] = fact_offres["full date"].astype(str) # Full date en string pour jointure
fact_offres = fact_offres.merge(dim_offres[["IdDemandeAuto","Key_offres"]], on=["IdDemandeAuto"], how='left') # Ajout de la clé technique Key_offres


# Calcul indicateur publication
fact_offres = fact_offres.merge(offres[["IdDemandeAuto","Ind dt publication de l'offre", "publication"]], left_on=["IdDemandeAuto", "full date"],right_on=["IdDemandeAuto","Ind dt publication de l'offre"], how='left')
fact_offres = fact_offres.groupby(["IdDemandeAuto","date key","full date","Key_offres"])["publication"].count().reset_index()

# Calcul indicateur first publication
stg_first_pub = fact_offres.loc[fact_offres["publication"]!=0]
stg_first_pub = stg_first_pub[["IdDemandeAuto","full date"]].groupby("IdDemandeAuto")["full date"].min().reset_index()
stg_first_pub["first publication"] = "1"
stg_first_pub["first publication"] = stg_first_pub["first publication"].astype(int)
fact_offres = fact_offres.merge(stg_first_pub, on=["IdDemandeAuto","full date"], how="left")


# Calcal indicateur clôture réelle

fact_cloture = fact_offres.merge(offres[["IdDemandeAuto","Ind clôture réelle", "clôture"]], left_on=["IdDemandeAuto", "full date"], right_on=["IdDemandeAuto","Ind clôture réelle"], how ='left')

fact_cloture = fact_cloture.groupby(["IdDemandeAuto","date key","full date","Key_offres"])["clôture"].count().reset_index()

fact_offres = fact_offres.merge(fact_cloture, on = ["IdDemandeAuto","date key","full date","Key_offres"], how='left')

# Ajout du champ fist virtual publication visant à identifier toutes les premières publication, et ajouter celles nécessaires pour les offres n'ayant qu'une date de clôture

fact_offres["first virtual publication"] = fact_offres["first publication"]

list_no_fpublication = fact_offres[["IdDemandeAuto", "first publication"]].groupby(["IdDemandeAuto"])["first publication"].count().reset_index()

list_no_fpublication = list_no_fpublication.loc[list_no_fpublication["first publication"]==0, "IdDemandeAuto"]

list_no_fpublication = list(list_no_fpublication)

list_one_cloture = fact_offres[["IdDemandeAuto", "clôture"]].groupby(["IdDemandeAuto"])["clôture"].sum().reset_index()

list_one_cloture = list_one_cloture.loc[list_one_cloture["clôture"]>= 1, "IdDemandeAuto"]

list_one_cloture = list(list_one_cloture)


first_date =  fact_temps.at[0, "full date"]
first_date = str(first_date)[:10]

fact_offres.loc[(fact_offres["IdDemandeAuto"].isin(list_no_fpublication)) & (fact_offres["full date"] == first_date) & (fact_offres["IdDemandeAuto"].isin(list_one_cloture)), "first virtual publication"]=1

# Ajout du témoin de statut

#1 Classement des offres par IdDeamndeAutot et full date
fact_offres = fact_offres.sort_values(by=["IdDemandeAuto", "full date"])
list_offre = list(fact_offres["IdDemandeAuto"].unique())

#2 Somme de publication et clôture
fact_offres["Sum"] = fact_offres["publication"] + fact_offres["clôture"]
for i, val in enumerate(fact_offres.iterrows()):
    if fact_offres.at[i, "first virtual publication"] == 1 and fact_offres.at[i, "publication"] == 0:
        fact_offres.at[i, "Sum"] += 1
        
#3 Cumul des sommes
fact_offres['Cumul'] = fact_offres.groupby(['IdDemandeAuto'])['Sum'].cumsum()

#4 Check pour chaque cumul si il est paire ou impore. Si cumul est impaire alors l'offre est publiée sinon non.
fact_offres["publié"] = fact_offres["Cumul"]
fact_offres["publié"] = fact_offres["publié"].apply(lambda x: x if x!=0 else 0)
fact_offres["publié"] = fact_offres["publié"].apply(lambda x: 0 if int(x)% 2== 0 else 1)

# Nombre d'ouverture d'offres
fact_ouv = fact_offres.merge(offres[["IdDemandeAuto","Ind date d'ouverture", "ouverture"]], left_on=["IdDemandeAuto", "full date"],right_on=["IdDemandeAuto","Ind date d'ouverture"], how='left')

fact_ouv = fact_ouv.groupby(["IdDemandeAuto","date key","full date","Key_offres"])["ouverture"].nunique().reset_index()

fact_offres = fact_offres.merge(fact_ouv, on = ["IdDemandeAuto","date key","full date","Key_offres"], how='left')

# Nombre de fermeture d'offres
fact_fer = fact_offres.merge(offres[["IdDemandeAuto","Ind date de fermeture", "fermeture"]], left_on=["IdDemandeAuto", "full date"],right_on=["IdDemandeAuto", "Ind date de fermeture"], how='left')

fact_fer = fact_fer.groupby(["IdDemandeAuto","date key","full date","Key_offres"])["fermeture"].nunique().reset_index()

fact_offres = fact_offres.merge(fact_fer, on = ["IdDemandeAuto","date key","full date","Key_offres"], how='left')

# Nombre d'ouverture virtuelles
fact_offres["virtual ouverture"] = fact_offres["ouverture"]

list_no_fouverture = fact_offres[["IdDemandeAuto", "ouverture"]].groupby(["IdDemandeAuto"])["ouverture"].sum().reset_index()

list_no_fouverture = list_no_fouverture.loc[list_no_fouverture["ouverture"]==0, "IdDemandeAuto"]

list_no_fouverture = list(list_no_fouverture)

list_one_fermeture = fact_offres[["IdDemandeAuto", "fermeture"]].groupby(["IdDemandeAuto"])["fermeture"].sum().reset_index()

list_one_fermeture = list_one_fermeture.loc[list_one_fermeture["fermeture"]>= 1, "IdDemandeAuto"]

list_one_fermeture = list(list_one_fermeture)

fact_offres.loc[(fact_offres["IdDemandeAuto"].isin(list_no_fouverture)) & (fact_offres["full date"] == first_date) & (fact_offres["IdDemandeAuto"].isin(list_one_fermeture)), "virtual ouverture"]=1

# Cas des ouvertures dans le passé n'étant pas présent dans table de fait  +  pas de date de clôture dans table de fait
nul_fermeture = offres.groupby(["IdDemandeAuto"])["Date de fermeture"].count().reset_index()
nul_fermeture = nul_fermeture.loc[nul_fermeture["Date de fermeture"]==0]
nul_fermeture = list(nul_fermeture["IdDemandeAuto"])

nul_ouverture = fact_offres.groupby(["IdDemandeAuto"])["virtual ouverture"].sum().reset_index()
nul_ouverture = nul_ouverture.loc[nul_ouverture["virtual ouverture"]==0]
nul_ouverture = list(nul_ouverture["IdDemandeAuto"])

nv_ouvert = [x for x in nul_fermeture if x in nul_ouverture ]
fact_offres.loc[fact_offres["IdDemandeAuto"].isin(nv_ouvert) & (fact_offres["full date"] == first_date), "virtual ouverture"] = 1

# Ajout du témoin de ouvert

#1 Somme de publication et clôture
fact_offres["Sum ouv"] = fact_offres["virtual ouverture"] + fact_offres["fermeture"]
        
#2 Cumul des sommes
fact_offres['Cumul ouv'] = fact_offres.groupby(['IdDemandeAuto'])['Sum ouv'].cumsum()

#3 Check pour chaque cumul si il est paire ou impore. Si cumul est impaire alors l'offre est publiée sinon non.
fact_offres["ouvert"] = fact_offres["Cumul ouv"]
fact_offres["ouvert"] = fact_offres["ouvert"].apply(lambda x: x if x!=0 else 0)
fact_offres["ouvert"] = fact_offres["ouvert"].apply(lambda x: 0 if int(x)% 2== 0 else 1)

#Cas des offres ouvertes mais non fermée


# indicateur nombre de candidatures
fact_candidature = fact_offres.merge(candidature[["IdDemandeAuto","Ind dépôt candidature", "candidature"]], left_on=["IdDemandeAuto", "full date"], right_on=["IdDemandeAuto","Ind dépôt candidature"], how ='left')
fact_candidature = fact_candidature.groupby(["IdDemandeAuto","date key","full date","Key_offres"])["candidature"].count().reset_index()
fact_offres = fact_offres.merge(fact_candidature, on = ["IdDemandeAuto","date key","full date","Key_offres"], how='left')

# indicateur cumul candidatures
fact_offres['Cumul candidature'] = fact_offres.groupby(['IdDemandeAuto'])['candidature'].cumsum()

# indicateurs sans nombre d'offres sans candidatures

fact_offres.loc[(fact_offres["Cumul candidature"]== 0) & ((fact_offres["publié"] ==1) | (fact_offres["ouvert"]==1)), "Nombre offres sans candidataures"] = 1


Wall time: 33.3 s


In [265]:

my_data.import_fact("fact_offres", fact_offres)

### Table de faits XXX

## Export des données vers le répertoire Transformed data

In [266]:
%%time
my_data.export_audit()
my_data.export()

Wall time: 1min 49s


In [267]:
#print(my_data.dim["Souhaits de formation"]["cspcode"].unique())